In [ ]:
import cv2
import pandas as pd
import os
from ultralytics import YOLO
from ultralytics.solutions import object_counter
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Initialize YOLO model
model = YOLO(r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrrfficAI\best.pt")
output_video =r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\Video\20_2_2024\tracked"
# Define video directory and Excel file path
videos_directory = r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\Video\20_2_2024\Split"
excel_file_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\Video\20_2_2024\Split\output.xlsx'

# Create an empty DataFrame to store all video data
all_videos_df = pd.DataFrame(columns=['Video', 'Lane counted car'])

# Loop through videos in the directory
for video_filename in os.listdir(videos_directory): 
    video_path = os.path.join(videos_directory, video_filename)
    
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"Error reading video file : {video_path}"
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Define line points
    line_points = [(800, 2000), (1800, 2000)]

    # Video writer
    video_writer = cv2.VideoWriter(r"C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\Video\20_2_2024\tracked\output.avi",
                        cv2.VideoWriter_fourcc(*'mp4v'),
                        fps,
                        (w, h))

    # Init Object Counter
    counter = object_counter.ObjectCounter()
    counter.set_args(view_img=True,  # Disable displaying the video
                    reg_pts=line_points,
                    classes_names=model.names,
                    draw_tracks=True)

    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video frame is empty or video processing has been successfully completed.")
            break
        tracks = model.track(im0, persist=True, show=False)
        cv2.line(im0, line_points[0], line_points[1], (255, 255, 0), 4)

        im0 = counter.start_counting(im0, tracks)
        video_writer.write(im0)

    cap.release()
    video_writer.release()

    # Append video data to DataFrame
    video_data = pd.DataFrame({'Video': [video_filename], 'Lane counted car': [counter.out_counts]})
    all_videos_df = pd.concat([all_videos_df, video_data], ignore_index=True)
    
# Save the aggregated DataFrame to Excel
all_videos_df.to_excel(excel_file_path, index=False)
print(f"Excel file '{excel_file_path}' has been created.")